In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
import time
import os
import copy
import shutil# **Testing**

In [2]:
len(os.listdir('/kaggle/input/pokemon-generation-one/dataset'))

150

In [3]:
basePath = '/kaggle/'

modelSave = basePath+'working/weights.pth'
data_dir = basePath+'input/pokemon-generation-one/dataset/'
working_dir = basePath+'working/'
model_file = '/kaggle/input/alexnet/alexnet.pth'

num_classes = 150
batch_size = 8
num_epochs = 50
input_size = 224

In [4]:
# Detect if we have a GPU available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [5]:
'''import zipfile
import os
for file_name in os.listdir('/content/'):
  if file_name.endswith('.zip'):
    with zipfile.ZipFile(file_name,'r') as zip_dir:
      zip_dir.extractall(path='/content/')'''

"import zipfile\nimport os\nfor file_name in os.listdir('/content/'):\n  if file_name.endswith('.zip'):\n    with zipfile.ZipFile(file_name,'r') as zip_dir:\n      zip_dir.extractall(path='/content/')"

In [6]:
# torch.utils.model_zoo.load_url('https://download.pytorch.org/models/alexnet-owt-4df8aa71.pth', model_dir='/kaggle/working')

In [7]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs=25):
    since = time.time()

    val_acc_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    outputs = model(inputs)
                    loss = criterion(outputs,labels)

                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history

In [8]:
def set_parameter_requires_grad(model):
    for param in model.parameters():
        param.requires_grad = False

In [9]:
def initialize_model(num_classes):
    alexnet = models.alexnet(pretrained=True)
    set_parameter_requires_grad(alexnet)
    num_ftrs = alexnet.classifier[6].in_features
    alexnet.classifier[6] = nn.Linear(num_ftrs,num_classes)
    return alexnet

In [10]:
def getTrainDataLoaders():
    # Data augmentation and normalization for training
    # Just normalization for validation
    data_transforms = {
        'train': transforms.Compose([
            transforms.RandomResizedCrop(input_size),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
        'val': transforms.Compose([
            transforms.Resize(input_size),
            transforms.CenterCrop(input_size),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
    }

    print("Initializing Datasets and Dataloaders...")

    # Create training and validation datasets
    image_datasets = {x: datasets.ImageFolder(os.path.join(working_dir, x), data_transforms[x]) for x in ['train', 'val']}
    # Create training and validation dataloaders
    dataloaders_dict = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True, num_workers=4) for x in ['train', 'val']}

    return dataloaders_dict

In [11]:
def getUpdatablePara(model):
    params_to_update = model.parameters()
    print("Params to learn:")
    params_to_update = []
    for name,param in model.named_parameters():
        if  param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
    return params_to_update

In [12]:
def split_train_test_data(test_size = 0.75):
    for pokemon_class in os.listdir(data_dir):
        if pokemon_class == 'dataset':
            continue
        print(pokemon_class)
        class_path = os.path.join(data_dir, pokemon_class)
        img_name_list = os.listdir(class_path)
        train_list = img_name_list[:int(test_size * len(img_name_list))]
        val_list = img_name_list[int(test_size * len(img_name_list)):]
        
        destination_folder = os.path.join(working_dir, 'train', pokemon_class)
        os.makedirs(destination_folder)            
        for img in train_list:
            source = os.path.join(class_path, img)
            destination = os.path.join(destination_folder, img)
            dest = shutil.copyfile(source, destination) 
        
        destination_folder = os.path.join(working_dir, 'val', pokemon_class)
        os.makedirs(destination_folder)
        for img in val_list:
            source = os.path.join(class_path, img)            
            destination = os.path.join(destination_folder, img)
            dest = shutil.copyfile(source, destination) 

In [13]:
def copy_model_to_cache():
    cache_dir = os.path.expanduser(os.path.join('~', '.torch'))
    if not os.path.exists(cache_dir):
        os.makedirs(cache_dir)
    models_dir = os.path.join(cache_dir, 'models')
    if not os.path.exists(models_dir):
        os.makedirs(models_dir)
    
    dest = shutil.copyfile(model_file, os.path.join(models_dir, os.path.basename(model_file)))

# **Prepare Model**

In [14]:
# copy_model_to_cache()

# **Prepare Train and Test Data**

In [15]:
split_train_test_data(0.8)

Metapod
Weepinbell
Pidgeot
Zubat
Voltorb
Dratini
Sandslash
Wartortle
Charmander
Flareon
Exeggcute
Nidorino
Spearow
Drowzee
Electabuzz
Poliwrath
Dewgong
Mankey
Jolteon
Weedle
Dragonair
Paras
Hypno
Shellder
Starmie
MrMime
Electrode
Weezing
Arcanine
Clefable
Omanyte
Ekans
Venonat
Kingler
Butterfree
Machop
Venusaur
Bellsprout
Cloyster
Scyther
Raichu
Meowth
Wigglytuff
Kabuto
Ponyta
Lickitung
Machoke
Magnemite
Onix
Grimer
Tentacruel
Parasect
Machamp
Squirtle
Haunter
Tangela
Arbok
Dragonite
Kangaskhan
Gengar
Poliwag
Vaporeon
Hitmonlee
Graveler
Primeape
Rhyhorn
Charmeleon
Charizard
Magneton
Ninetales
Mew
Gloom
Hitmonchan
Slowbro
Rhydon
Pidgey
Cubone
Marowak
Seel
Victreebel
Golem
Horsea
Lapras
Jigglypuff
Pinsir
Goldeen
Diglett
Beedrill
Growlithe
Chansey
Tauros
Kakuna
Exeggutor
Tentacool
Nidoking
Ditto
Abra
Slowpoke
Clefairy
Poliwhirl
Zapdos
Venomoth
Aerodactyl
Fearow
Nidorina
Muk
Caterpie
Blastoise
Eevee
Geodude
Alakazam
Doduo
Snorlax
Gastly
Nidoqueen
Raticate
Articuno
Dugtrio
Vulpix
Oddish
Gol

In [16]:
# count = 0
# for pokemon_class in os.listdir(data_dir):
#     if pokemon_class == 'dataset':
#         continue
#     class_path = os.path.join(data_dir, pokemon_class)
#     class_imgs = len(os.listdir(class_path))
#     train_imgs = len(os.listdir(os.path.join(working_dir, 'train', pokemon_class)))
#     val_imgs = len(os.listdir(os.path.join(working_dir, 'val', pokemon_class)))
#     if(class_imgs != (train_imgs + val_imgs)):
#         print(pokemon_class, class_imgs, train_imgs, val_imgs)
#         count = count + 1
# print(count)

# **Training**

In [17]:
alexnet = initialize_model(num_classes)
alexnet = alexnet.to(device)
#print(alexnet)

Downloading: "https://download.pytorch.org/models/alexnet-owt-4df8aa71.pth" to /root/.cache/torch/checkpoints/alexnet-owt-4df8aa71.pth


In [18]:
dataloaders = getTrainDataLoaders()

Initializing Datasets and Dataloaders...


In [19]:
params_to_update = getUpdatablePara(alexnet)

Params to learn:
	 classifier.6.weight
	 classifier.6.bias


In [20]:
# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(params_to_update, lr=0.0001, momentum=0.9)

#loss function 
criterion = nn.CrossEntropyLoss()

alexnet, hist = train_model(alexnet, dataloaders, criterion, optimizer_ft, num_epochs=num_epochs)

torch.save(alexnet.state_dict(), modelSave)

Epoch 0/49
----------


/opt/conda/lib/python3.6/site-packages/PIL/Image.py:969: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')


train Loss: 4.1139 Acc: 0.1715


/opt/conda/lib/python3.6/site-packages/PIL/Image.py:969: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')
/opt/conda/lib/python3.6/site-packages/PIL/Image.py:969: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')


val Loss: 3.2206 Acc: 0.3088

Epoch 1/49
----------


...

val Loss: 1.7568 Acc: 0.5866

Epoch 49/49
----------


/opt/conda/lib/python3.6/site-packages/PIL/Image.py:969: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')


train Loss: 1.2130 Acc: 0.6950


/opt/conda/lib/python3.6/site-packages/PIL/Image.py:969: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')
/opt/conda/lib/python3.6/site-packages/PIL/Image.py:969: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')


val Loss: 1.7660 Acc: 0.5829

Training complete in 99m 23s
Best val Acc: 0.593422


# **Testing**

In [21]:
'''alexnet = initialize_model(num_classes)

alexnet.load_state_dict(torch.load(modelSave))
alexnet.eval()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
alexnet = alexnet.to(device)
'''

'alexnet = initialize_model(num_classes)\n\nalexnet.load_state_dict(torch.load(modelSave))\nalexnet.eval()\ndevice = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")\nalexnet = alexnet.to(device)\n'